In [57]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [58]:
data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_5\data\insurance_claims.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Admin\AppData\Local\Temp\ipykernel_27848\1517680743.py:1: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_5\data\insurance_claims.csv')


In [59]:
dropc_list = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year', '_c39',
           'age','total_claim_amount']
dropr_list = [578]
df = data.copy()

df.drop(columns=dropc_list, inplace=True)
df.drop(index=dropr_list, inplace=True)
df['fraud_reported'] = df['fraud_reported'].replace({'Y':1, 'N':0})
target = df.pop('fraud_reported')
df.replace('?', np.nan, inplace = True)
df['collision_type'] = df['collision_type'].fillna(df['collision_type'].mode()[0])
df['property_damage'] = df['property_damage'].fillna(df['property_damage'].mode()[0])
df['police_report_available'] = df['police_report_available'].fillna(df['police_report_available'].mode()[0])
o_col = df.select_dtypes(include = ['object'])
num_df = df.select_dtypes(include = ['int64'])
o_col = pd.get_dummies(o_col)


scaler = StandardScaler()
name = num_df.columns
num_df = scaler.fit_transform(num_df)
num_df = pd.DataFrame(num_df, columns = name)
df = pd.concat([num_df, o_col], axis = 1)
df = df.dropna()
target = target.dropna()
target.drop(axis  = 0, index = 998,inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_27848\1023905105.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['fraud_reported'] = df['fraud_reported'].replace({'Y':1, 'N':0})


In [60]:
target.value_counts()

fraud_reported
0    751
1    247
Name: count, dtype: int64

In [61]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(1, 10, 2)) + [None],
    'min_samples_leaf': range(1, 10, 2),
    'min_samples_split': range(1, 10, 2),
}

In [62]:
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, random_state = 42)

In [63]:
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 5)

In [64]:
#grid_search.fit(X_train, y_train)

In [65]:
#grid_search.best_estimator_

In [66]:
dt = DecisionTreeClassifier(
                            max_depth = 1,
                            min_samples_leaf= 3, 
                            min_samples_split = 3, 
                            random_state = 42,)

In [67]:
dt.fit(X_train, y_train)
print(f1_score(dt.predict(X_train), y_train))
print(f1_score(dt.predict(X_test), y_test))

0.6405867970660146
0.631578947368421


In [68]:
rf = RandomForestClassifier(criterion= 'entropy',
                            min_samples_leaf= 1, 
                            min_samples_split = 10, 
                            n_estimators= 82, 
                            random_state = 42,
                            class_weight = 'balanced_subsample',
)
rf.fit(X_train, y_train)
print(f1_score(rf.predict(X_train), y_train))
print(f1_score(rf.predict(X_test), y_test))

0.908641975308642
0.6181818181818182


In [69]:
c = np.array(rf.feature_importances_)
features = np.array(df.columns) 

feat_df = pd.DataFrame({'feature': features, 'importance': c})

top5 = feat_df.sort_values(by='importance', ascending=False).head(5)
print(top5)

                           feature  importance
51  incident_severity_Major Damage    0.179753
10                  property_claim    0.063123
11                   vehicle_claim    0.060536
9                     injury_claim    0.058690
0               months_as_customer    0.054954


In [70]:
c = np.array(dt.feature_importances_)
features = np.array(df.columns)  

feat_df = pd.DataFrame({'feature': features, 'importance': c})

top5 = feat_df.sort_values(by='importance', ascending=False).head(5)
print(top5)

                           feature  importance
51  incident_severity_Major Damage         1.0
1                policy_deductable         0.0
0               months_as_customer         0.0
3                    capital-gains         0.0
4                     capital-loss         0.0


In [71]:
df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,...,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police,property_damage_NO,property_damage_YES,police_report_available_NO,police_report_available_YES
0,1.081258,-0.222499,-0.479771,1.012150,0.954763,-0.955411,-0.823305,0.009764,0.463400,-0.189102,...,False,False,False,False,False,True,False,True,False,True
1,0.211081,1.411889,1.696847,-0.900939,0.954763,-0.523791,-0.823305,-1.209559,-1.337910,-1.363058,...,False,False,False,False,False,True,True,False,True,False
2,-0.606885,1.411889,1.696847,0.358900,0.954763,-0.667664,1.139659,1.229087,1.364055,0.054704,...,False,False,False,False,False,True,True,False,True,False
3,0.454731,1.411889,2.132170,0.854221,-1.266566,-0.955411,-0.823305,0.009764,0.463400,-0.223931,...,False,False,False,False,False,True,True,False,True,False
4,0.211081,-0.222499,2.132170,1.467989,-0.682755,1.202688,-0.823305,-1.209559,-0.437255,-1.256521,...,False,False,False,False,False,False,True,False,True,False
